# Problem statement
__Prediction Question:__ <br>
How accurately can I predict the price of a house, given the values of all variables? 

In [ ]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [ ]:
# Load the data

df = pd.read_csv('/kaggle/input/others/House_Price.csv')
df.head()

In [ ]:
# Check the shape of the dataset

print(df.shape)
print("Number of rows: ",df.shape[0])
print("Number of columns: ",df.shape[1])

In [ ]:
df.info()

In [ ]:
 df.describe()

In [ ]:
df.skew()

##### Univariate Analysis On the Continuous variables:

In [ ]:
df_continuous = [col for col in df.select_dtypes(include=[np.number])]
print("Continuous variables: \n",df_continuous)

In [ ]:
sns.jointplot(x='crime_rate',y='price',data=df)

As "crime_rate" increases the price drops. 

In [ ]:
sns.jointplot(x='resid_area',y='price',data=df)

In [ ]:
sns.jointplot(x='air_qual',y='price',data=df)

In [ ]:
sns.jointplot(x='room_num',y='price',data=df)

In [ ]:
sns.jointplot(x='teachers',y='price',data=df)

In [ ]:
sns.jointplot(x='n_hot_rooms',y='price',data=df)

In [ ]:
sns.jointplot(x='rainfall',y='price',data=df)

- As the crime_rate increase the "price" decreases. There might be negative colinearity between these features.<br>
- As the "room_num" increases the "price" increases too.<br>
- In "n_hot_rooms" variable, most of the data lies between 1 & 20 but there are 2 outliers.<br>
- "rainfall" variable has most of the data between 20 & 60 but there is one outlier below 10 which might be a sampling error or calculation error. 

##### Univariate Analysis On the Categorical variables:

In [ ]:
df_categorical = [col for col in df.select_dtypes(include=[np.object])]
print("Categorical variables: \n",df_categorical)

In [ ]:
sns.countplot(x='airport',data=df)

In [ ]:
sns.countplot(x='waterbody',data=df)

In [ ]:
sns.countplot(x='bus_ter',data=df)

__Outlier Treatment__

In [ ]:
np.percentile(df['n_hot_rooms'],[99])

In [ ]:
upper_limit = np.percentile(df['n_hot_rooms'],[99])[0]
upper_limit

In [ ]:
df[(df['n_hot_rooms']>upper_limit)]

In [ ]:
# n=3

df['n_hot_rooms'][(df['n_hot_rooms']>3*upper_limit)]=3*upper_limit

In [ ]:
lower_limit = np.percentile(df.rainfall,[1])[0]
lower_limit

In [ ]:
df[(df.rainfall<lower_limit)]

In [ ]:
df.rainfall[(df.rainfall<0.3*lower_limit)]=0.3*lower_limit

In [ ]:
sns.jointplot(x='crime_rate', y='price',data=df)

In [ ]:
df.describe()

In [ ]:
df.info()

 __Missing Value Treatment__

In [ ]:
df["n_hos_beds"]=df["n_hos_beds"].fillna(df["n_hos_beds"].mean())

In [ ]:
df.info()

__Bivariate Analysis__

In [ ]:
sns.jointplot(x = 'crime_rate', y ='price',data=df)

In [ ]:
df['crime_rate'] = np.log(1+df['crime_rate'])

In [ ]:
sns.jointplot(x = 'crime_rate', y ='price',data=df)

In [ ]:
df['avg_dist'] = (df['dist1']+df['dist2']+df['dist3']+df['dist4'])/4

In [ ]:
df.describe()

In [ ]:
df.drop(['dist1','dist2','dist3','dist4','bus_ter'],axis=1,inplace=True)

In [ ]:
# create dummy variables

df = pd.get_dummies(df)

In [ ]:
df.head()

In [ ]:
df.drop(['airport_NO','waterbody_None'],inplace=True,axis=1)

In [ ]:
df.head(1)

In [ ]:
df.corr()

In [ ]:
df.drop('parks',inplace=True,axis=1)

In [ ]:
df.head(1)

__Linear Regression model__

In [ ]:
import statsmodels.api as sn

In [ ]:
X = sn.add_constant(df['room_num'])

In [ ]:
lm= sn.OLS(df['price'],X).fit()

In [ ]:
lm.summary()

In [ ]:
# another method

from sklearn.linear_model import LinearRegression

In [ ]:
y = df['price']

In [ ]:
X = df[['room_num']]

In [ ]:
lm2 = LinearRegression()

In [ ]:
lm2.fit(X,y)

In [ ]:
print('intercept: ',lm2.intercept_,'\ncoefficient: ',lm2.coef_)

In [ ]:
lm2.predict(X)

Predicted house prices.

In [ ]:
sns.jointplot(x=df['room_num'],y=df['price'],data=df,kind='reg')

In [ ]:
X_multi = df.drop('price',axis=1)

In [ ]:
y_multi = df['price']

In [ ]:
y_multi.head()

In [ ]:
X_multi_cons = sn.add_constant(X_multi)

In [ ]:
X_multi_cons.head()

In [ ]:
lm_multi = sn.OLS(y_multi,X_multi_cons).fit()

In [ ]:
lm_multi.summary()

In [ ]:
lm3 = LinearRegression()

In [ ]:
lm3.fit(X_multi,y_multi)

In [ ]:
print(lm3.intercept_,lm3.coef_)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_multi,y_multi,test_size = 0.2,random_state=0)

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
lm_a = LinearRegression()

In [ ]:
lm_a.fit(X_train,y_train)

In [ ]:
y_test_a = lm_a.predict(X_test)

In [ ]:
y_train_a = lm_a.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test,y_test_a)

In [ ]:
r2_score(y_train,y_train_a)

With the Base model we are getting a accuracy of 54%.